In [82]:
import os
import glob
import IPython.display as ipd
from pathlib import Path

%matplotlib inline
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Subtract
from keras.optimizers import Adam, SGD
from keras.utils import plot_model
from keras.regularizers import l2
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from keras.initializers import RandomNormal

from tqdm.notebook import tqdm, trange

In [83]:
DATA_DIR = Path('./frgc32x32/')
image_paths = [x.name for x in DATA_DIR.glob('*')]

paths_all_labels = []
for image in tqdm(image_paths, desc='finding images'):
    x = image.split("d", 1)
    paths_all_labels.append((x[0], './frgc32x32/'+image))

cat_list = [] #list of all unique labels
images = []   #list of all images
labels = []   #list of all labels on same idx as image

for (label, path) in tqdm(paths_all_labels, desc='loading images'):
    img = cv2.imread(path, 0)
    sample = cv2.resize(img, (32,32))
    sample = np.expand_dims(img, -1)
    images.append(sample)
    labels.append(label)
    if label not in cat_list:
        cat_list.append(label)

cat_list = np.asarray(cat_list)
images = np.asarray(images)
labels = np.asarray(labels)

print(images.shape, labels.shape, cat_list.shape)

finding images:   0%|          | 0/12307 [00:00<?, ?it/s]

loading images:   0%|          | 0/12307 [00:00<?, ?it/s]

(12307, 32, 32, 1) (12307,) (568,)


In [84]:
train_test_split = 0.7
train_size = int(len(cat_list) * train_test_split)
test_size = len(cat_list) - train_size
print(train_size, 'classes for training and', test_size, ' classes for testing')

train_idx = 0
while labels[train_idx+1] != cat_list[train_size]:
    train_idx += 1
train_idx += 1

img_train = images[:train_idx]
lbl_train = labels[:train_idx]
cat_train = cat_list[:train_size]

img_val = images[train_idx:]
lbl_val = labels[train_idx:]
cat_val = cat_list[train_size:]

print('X&Y shape of training data :',img_train.shape, 'and', lbl_train.shape, cat_train.shape)
print('X&Y shape of testing data :' , img_val.shape, 'and', lbl_val.shape, cat_val.shape)

397 classes for training and 171  classes for testing
X&Y shape of training data : (9947, 32, 32, 1) and (9947,) (397,)
X&Y shape of testing data : (2360, 32, 32, 1) and (2360,) (171,)


In [85]:
def get_batch(batch_size=64):
    temp_img = img_train
    temp_cat_list = cat_train
    start = 0
    end = train_size
    batch_img = []

    batch_lbl = np.zeros(batch_size)
    batch_lbl[int(batch_size/2):] = 1
#     TODO implement batches

In [88]:
input_shape = (32,32,1)
left_input = Input(input_shape)
right_input = Input(input_shape)

W_init = RandomNormal(mean = 0.0, stddev = 1e-2)
b_init = RandomNormal(mean = 0.5, stddev = 1e-2)

model = Sequential([
    keras.layers.Conv2D(64, (6,6), activation='relu', input_shape=input_shape, kernel_initializer=W_init, bias_initializer=b_init, kernel_regularizer=l2(2e-4)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(128, (4,4), activation='relu', kernel_initializer=W_init, bias_initializer=b_init, kernel_regularizer=l2(2e-4)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='sigmoid', kernel_initializer=W_init, bias_initializer=b_init)
])

encoded_l = model(left_input)
encoded_r = model(right_input)

subtracted = keras.layers.Subtract()([encoded_l, encoded_r])
prediction = Dense(1, activation='sigmoid', bias_initializer=b_init)(subtracted)
siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)

optimizer= Adam(learning_rate=0.0006)
siamese_net.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
def nway_one_shot(model, n_way, n_val):
    pass





In [90]:
epochs = 10000
n_way = 10
n_val = 50
batch_size = 64

loss_list = []
accuracy_list = []

for epoch in tqdm(range(1, epochs), desc='training'):
    pass

training:   0%|          | 0/9999 [00:00<?, ?it/s]